# 1. Example to grap images from a webcam
(but this only gives you an rgb, not depth)
hit 'q' in the window to stop

In [1]:
import cv2 as cv

In [2]:
cap = cv.VideoCapture(0)
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    # Our operations on the frame come here
    img = frame
    #img = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Display the resulting frame
    if len(frame)>0: cv.imshow('img', img)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

[ WARN:0@0.747] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@0.747] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


TypeError: object of type 'NoneType' has no len()

In [3]:
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()

# 2. Example to convert simulated RGB-D images into point clouds

In [ ]:
import os
os._exit(0)

In [1]:
import cv2 as cv
import numpy as np
from robotic import ry
import time
print(cv.__version__)

4.7.0


In [2]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandasTable.g'))

obj = C.addFrame('obj')
obj.setPose('t(0. 0.1 0.8)')
obj.setShape(ry.ST.ssBox, size=[.05,.05,.05,.005])
obj.setColor([1,.0,0])
obj.setMass(.1)
obj.setContact(True)
C.view()

0

In [3]:
bot = ry.BotOp(C, False)
bot.home(C)

In [4]:
fxypxy = bot.getCameraFxypxy("camera")
print(fxypxy)
cameraFrame = C.getFrame("camera")

[rai] bot.cpp:getCameraFxypxy:309(0) creating camera sensor 'camera'
[322.19998169 322.19998169 320.         180.        ]


In [5]:
points = []
tau = .01

for t in range(30):
    time.sleep(0.1)

    #grab sensor readings from the simulation
    [rgb, depth] = bot.getImageAndDepth('camera')  #we don't need images with 100Hz, rendering is slow
            
    if len(rgb)>0: cv.imshow('OPENCV - rgb',
                                 cv.cvtColor(rgb, cv.COLOR_BGR2RGB))
    if len(depth)>0: cv.imshow('OPENCV - depth', 0.5* depth)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break
        
    bot.sync(C)

In [6]:
cv.destroyAllWindows()
del bot
del C

[rai] bot.cpp:~BotOp:87(0) shutting down BotOp
[rai] simulation.cpp:~BotThreadedSim:60(0) shutting down SimThread
[rai] simulation.cpp:~Simulation:153(0) shutting down Simulation


# 3. example to use multiple camera attached to different robot frames

In [ ]:
import os
os._exit(0)

In [13]:
from robotic import ry
import numpy as np

In [14]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandasTable.g'))
# change the position of the central sensor
f = C.frame("camera")
f.setPosition(f.getPosition()+[0,0,.5])
# add a frame for the additional camera
f = C.addFrame("r_gripperCamera", "r_gripper")
f.setShape(ry.ST.camera, [.1])
f.addAttributes({'focalLength':0.895, 'width':640., 'height':360.})

In [15]:
bot = ry.BotOp(C, False)
bot.home(C)

In [16]:
img, _ = bot.getImageAndDepth('camera')

[rai] bot.cpp:getImageAndDepth:294(0) creating camera sensor 'camera'


In [17]:
img, _ = bot.getImageAndDepth('r_gripperCamera')

[rai] bot.cpp:getImageAndDepth:294(0) creating camera sensor 'r_gripperCamera'


In [18]:
del bot
del C

[rai] bot.cpp:~BotOp:87(0) shutting down BotOp
[rai] simulation.cpp:~BotThreadedSim:60(0) shutting down SimThread
[rai] simulation.cpp:~Simulation:153(0) shutting down Simulation
